<a href="https://colab.research.google.com/github/PrajwalPrashanth/EIP2_Phase1/blob/master/Session%204/Documented_9_4L_DNST_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Genral Summary 

* Used Google Colab for GPU 
* Ran with 5 different Accounts to (4 windows in one virtual desktop to test different models and 1 for experimentation of ideas that pops or something written in a blog)
* Some of the models and logs were lost when the browser crashed and output cells were gone, i'll add the links below for other model which did'nt go wrong


![alt text](https://drive.google.com/file/d/1mIgtVtBouHEZub9kMNJo2TrwFsqUAvjt/view?usp=sharing)

## Training Summary 

* This is the folow-up of my first submission which was done on Oct 22nd 11:53pm (5 days after my 4th session) had then validation accuracy 92.8% and moved upto 93.41% now.

* This model has total params of 940k and it's using compression only not bottleneck

* For first 50 epochs i have twice the steps per epoch as i had read in the keras doc the generator will keep on generating the images, so i made it twice the no. of images , as i read in a medium post that will help , tried thrice didnt help much and it took a lot of time . as im not sure how this will be graded i have done other model with 50k only i.e same no. of steps for as much as 50k images.

**Model**

* Went through blogs to understand about densenet as i was not getting much out of the paper, after some understanding gotto know about the terminologies and hit the paper once again.
* The filters for each layer in dense block were too low in the default code u had shared as it was multiplied with compression so went ahead and replaced with a separate variable as growth_rate
* In some Models i have implemented the actual bottleneck like mentioned in the paper
* Tried different variations out of it and did experimenations.
* Using only 3 pairs of dense and transistion block

**Batch Size**

* Ranged from 32 to 256. In most cases 64 was the best.

**Learning Rate**

* This was the key tuning parameter to reach the mark
* Was using Cyclic Learning Rate
* Range 0.1 to 0.2 in the beginning for faster convergence
* Then kept dividing by 10 after some epochs
* In Some places i have used 0.075 to 0.15 based on graph on cifar-10 lr vs loss graph which was shown in CLR github repo.
* Also in some places at the end have used static learning rate.




## Models Summary 



1.   This Submission | val_acc: 93.41 | 109 epochs | 940k total params | twice steps per epoch | Densenet compression only


---


2.   Same type like the submitted file Compression only model | val_acc 93.43 | too shaby so didnt submit this one | https://colab.research.google.com/drive/1-ab24n43TGBEEGxig6Rj6-3qP4BeqXPR


---


3.  Replica of Densenet BC-100-12 | aprrox 793k total params | val_acc:93.24 |94  epochs | didnt use droput from the start | https://colab.research.google.com/drive/1ib3PGTNW4SlPGVqY7bHf0xhrQ6OvmCRs

---


4.  Like Densenet bottleneck compression but with more layers inside denseblock | 963k | val_acc:93.00 | 121 epochs | didnt drop augmentation completely | https://colab.research.google.com/drive/1RaI3DO_V74PFkKFJOl4EiuFOkP3Rvyf3

---


5. Tried wide densenet too though theri training time were almost half of this didnt find good accuracy , and did'nt tune it further.


---





## Initial imports and cifar-10 data

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170500096/170498071 [==============================] - 6s 0us/step


## Connecting to drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

## Image Augmentation

* Geometric transformations for the model to learn better on object's features than all over the image.
* Zoom and shifts to make it like it has been cropped

In [0]:
datagen = ImageDataGenerator(
                horizontal_flip = True,
    width_shift_range = 0.1,height_shift_range = 0.1,zoom_range=0.1                                  )

## Checkpoint

To get along with colab to save weights when they improve


In [0]:
filepath="/content/gdrive/My Drive/test_9.4L/ep:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

## Cyclic learning rates

* Using Cyclic learning rates to converge faster in the begining and also not to get stuck in local minima which can happen using a static learning rate.

In [9]:
!rm -rf clr_callback.py*
!wget https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
from clr_callback import *

--2018-11-01 03:37:29--  https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326 (5.2K) [text/plain]
Saving to: ‘clr_callback.py’

clr_callback.py     100%[===================>]   5.20K  --.-KB/s    in 0s      

2018-11-01 03:37:29 (42.5 MB/s) - ‘clr_callback.py’ saved [5326/5326]



## Model block Fucntions
* i got confused in the begining as the default code had used terminolgies like bottleneck, growth_rate differently as i was comparing with densenet paper initially.

* got a blog which made it simpler for me to understand those terminolgies better

* in default code num_filter * compression was acting like growth_rate went ahead and changed that to a separate variable to have more growth rate, which for me made a sensible change as there were only 6 channels added in the default code

* num_filter was now global which only affected the first conv layer

* Replaced global average pooling instead of flatten

* Actually this model does'nt have bottle neck to reduce parameteres in dense block , it only densenet like model with compression 


In [0]:
# Dense Block
def add_denseblock(input, dropout_rate = 0.2):
    global compression
    global num_filter
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(growth_rate, (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        num_filter = num_filter + growth_rate
        temp = concat
        
    return temp

In [0]:
def add_transition(input, dropout_rate = 0.2):
    global compression
    global num_filter
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    num_filter = num_filter * compression
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    GAvgPooling = GlobalAveragePooling2D()(relu)
    output = Dense(num_classes, activation='softmax', use_bias=False)(GAvgPooling)
    
    return output

## Parameters to build the model

* introduced growth_rate as 12
* made l(no. of layers of bn-relu-conv in one denseblock) as 15 as it was crossing the constraint with l=16 of 1M parameters 
* num_filter as 24 i felt 12 was pretty low, also for densenet bc 100 12 made use of 24 for first conv, made me make it 24

In [0]:
# Hyperparameters
num_classes = 10
l = 15
num_filter = 24
compression = 0.5
dropout_rate = 0
growth_rate = 12

## Model

* Stuck on to 3 denseblocks, by removing other last block which was added in the default code
* Ended with  
             Total params: 939,576
             Trainable params: 922,392
              Non-trainable params: 17,184

In [0]:
#model
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, dropout_rate)
First_Transition = add_transition(First_Block, dropout_rate)

Second_Block = add_denseblock(First_Transition, dropout_rate)
Second_Transition = add_transition(Second_Block, dropout_rate)

Third_Block = add_denseblock(Second_Transition, dropout_rate)
Third_Transition = add_transition(Third_Block, dropout_rate)

output = output_layer(Third_Block)


In [15]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

## Opening in a new tab logs were not present unless went to open in playground , opening in a private window displayed the logs. Still if the logs are not available download zip file to get ipynb file in this gist link

https://gist.github.com/PrajwalPrashanth/2a813f1f345be3a1e0ecfd8952c06efa

## Training

As i was saving best val_acc for checkpoints i have used and with colab crashing and me leaving it to run in the background it would have been disconnected so have mentinoed effective epochs and the actual run in the brackets so there's no confusion.

### Effective Training Epochs (1 --> 46) (_run (1 --> 50)ends at 46_)

* **CLR** | **range(0.1,0.2)** | **step_size=2000** 

* Have used twice steps_per_epoch for first 50 epochs 

* Again with training batch size 64 was giving good results initially

* **max val_acc 90.46 in 46th epoch**

In [0]:
#model.load_weights('/content/gdrive/My Drive/6l_clr/file_name-77-0.84.hdf5')
clr_t = CyclicLR(mode='triangular2',base_lr=0.1,max_lr=0.2,step_size=2000)
callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])


In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    epochs=epochs,
                    steps_per_epoch = len(x_train)/32,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### Effective Training Epochs (47 --> 50) (_Run (1 --> 19)ends at 4_ )

* **CLR** | **range(0.0075,0.015)** | **step_size=1000**  

* Batch size 64

* **max val_acc 91.92 in 50th(here 4th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:045-val_acc:0.905.hdf5')
clr_t = CyclicLR(mode='triangular',base_lr=0.0075,max_lr=0.015,step_size=1000)
callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])


In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    epochs=30,
                    verbose=1,
                    steps_per_epoch = len(x_train)/32,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### Effective Training Epochs (51 --> 60) (_Run (1 --> 10)ends at 10_ )

* **CLR** | **range(0.00075,0.0015)** | **step_size=500**  

* Batch size 32

* **max val_acc 91.97 in 60th(here 10th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:004-val_acc:0.919.hdf5')
clr_t = CyclicLR(mode='triangular',base_lr=0.00075,max_lr=0.0015,step_size=500)
callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])


In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### Effective Training Epochs (61 --> 63) (_Run (1 --> 10)ends at 3_ )

* **CLR** | **range(0.00075,0.0015)** | **step_size=500**  

* Batch size 32

* **max val_acc 92.24 in 63rd(here 3rd) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:49+010-val_acc:0.920.hdf5')
clr_t = CyclicLR(mode='triangular',base_lr=0.000075,max_lr=0.00015,step_size=500)
callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### ** Dropped Augmentation from 64th epoch onwards **

* Used model.fit() to drop augmentation as i was not concatenating the origianal dataset and was only passing the augmented images the model was missing on some features i think so i dropped it at the end and after breaching 92% mark.  

### Effective Training Epochs (64 --> 73) (_Run (1 --> 10)ends at 9_ )

* lr .0001  


* Batch size 64

* **max val_acc 92.77 in 73rd(here 9th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:59+003-val_acc:0.922.hdf5')
#clr_t = CyclicLR(mode='triangular',base_lr=0.000075,max_lr=0.00015,step_size=500)
#callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(.0001),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### Effective Training Epochs (74 --> 84) (_Run (1 --> 10)ends at 10_ )

* lr .0001  

* Batch size 128

* **max val_acc 92.80 in 83rd(here 9th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:62+009-val_acc:0.928.hdf5')
#clr_t = CyclicLR(mode='triangular',base_lr=0.000075,max_lr=0.00015,step_size=500)
#callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(.0001),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### Effective Training Epochs (85 --> 88) (_Run (1 --> 10)ends at 4_ )

* weights carried from last run as there was no disconnection 
* lr .00001  


* Batch size 256

* **max val_acc 92.81 in 88nd(here 4th) epoch**

In [0]:
#model.load_weights('/content/gdrive/My Drive/2times:ep:62+009-val_acc:0.928.hdf5')
#clr_t = CyclicLR(mode='triangular',base_lr=0.000075,max_lr=0.00015,step_size=500)
#callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(.00001),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

### **Dropped dropout from 89th epoch**

* Dropped dropout too at the end so that it can learn some specific features too , i used dropout in the begining so that the model wont overfit and dropping it only at the end.

* Made droput value 0 and ran the model again .

### Effective Training Epochs (89 --> 99) (_Run (1 --> 10)ends at 10_ )

* lr .001  

* Batch size 64

* **max val_acc 93.25 in 99th(here 10th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/2times:ep:81+004-val_acc:0.928.hdf5')
#clr_t = CyclicLR(mode='triangular',base_lr=0.000075,max_lr=0.00015,step_size=500)
#callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(.001),
              metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train, batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 320s 6ms/step - loss: 0.0503 - acc: 0.9829 - val_loss: 0.2344 - val_acc: 0.9293

Epoch 00001: val_acc did not improve from 0.92960
Epoch 2/10
50000/50000 [==============================] - 311s 6ms/step - loss: 0.0474 - acc: 0.9835 - val_loss: 0.2321 - val_acc: 0.9285

Epoch 00002: val_acc did not improve from 0.92960
Epoch 3/10
50000/50000 [==============================] - 311s 6ms/step - loss: 0.0458 - acc: 0.9838 - val_loss: 0.2307 - val_acc: 0.9297

Epoch 00003: val_acc improved from 0.92960 to 0.92970, saving model to /content/gdrive/My Drive/test_9.4L/ep:003-val_acc:0.930.hdf5
Epoch 4/10
50000/50000 [==============================] - 310s 6ms/step - loss: 0.0444 - acc: 0.9849 - val_loss: 0.2287 - val_acc: 0.9301

Epoch 00004: val_acc improved from 0.92970 to 0.93010, saving model to /content/gdrive/My Drive/test_9.4L/ep:004-val_acc:0.930.hdf5
Epoch 5/10
50000/50000 [======

### Effective Training Epochs (100 --> 104) (_Run (1 -->5)ends at 4_ )

* **CLR | 0.001 to 0.01 | Step size:500**

* Batch size 64

* **max val_acc 93.39 in 104th(here 10th) epoch**

In [0]:
model.load_weights('/content/gdrive/My Drive/test_9.4L/ep:010-val_acc:0.932.hdf5')
clr_t = CyclicLR(mode='triangular',base_lr=0.001,max_lr=0.01,step_size=500)
#callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])

In [44]:
model.fit(x_train, y_train, batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 326s 7ms/step - loss: 0.0396 - acc: 0.9870 - val_loss: 0.2259 - val_acc: 0.9319

Epoch 00001: val_acc improved from -inf to 0.93190, saving model to /content/gdrive/My Drive/test_9.4L/ep:001-val_acc:0.932.hdf5
Epoch 2/5
50000/50000 [==============================] - 311s 6ms/step - loss: 0.0362 - acc: 0.9881 - val_loss: 0.2238 - val_acc: 0.9331

Epoch 00002: val_acc improved from 0.93190 to 0.93310, saving model to /content/gdrive/My Drive/test_9.4L/ep:002-val_acc:0.933.hdf5
Epoch 3/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.0335 - acc: 0.9891 - val_loss: 0.2246 - val_acc: 0.9324

Epoch 00003: val_acc did not improve from 0.93310
Epoch 4/5
50000/50000 [==============================] - 310s 6ms/step - loss: 0.0307 - acc: 0.9908 - val_loss: 0.2239 - val_acc: 0.9339

Epoch 00004: val_acc improved from 0.93310 to 0.93390, saving model to /content/gdrive/

### Effective Training Epochs (105 --> 109) (_Run (1 --> 5)ends at 4_ )

* **CLR | 0.0001 to 0.001 | Step Size : 500**

* Batch size 64

* **max val_acc 93.41 in 109th(here 4th) epoch**

In [0]:
filepath="/content/gdrive/My Drive/test_9.4L/ep:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
#model.load_weights('/content/gdrive/My Drive/test_9.4L/ep:010-val_acc:0.932.hdf5')
clr_t = CyclicLR(mode='triangular',base_lr=0.0001,max_lr=0.001,step_size=500)
callbacks_list.append(clr_t)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
            optimizer=SGD(),
              metrics=['accuracy'])

In [46]:
model.fit(x_train, y_train, batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
  128/50000 [..............................] - ETA: 45:33 - loss: 0.0124 - acc: 1.0000  

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.361691). Check your callbacks.
  % delta_t_median)


50000/50000 [==============================] - 320s 6ms/step - loss: 0.0262 - acc: 0.9928 - val_loss: 0.2246 - val_acc: 0.9337

Epoch 00001: val_acc improved from -inf to 0.93370, saving model to /content/gdrive/My Drive/test_9.4L/ep:001-val_acc:0.934.hdf5
Epoch 2/5
50000/50000 [==============================] - 314s 6ms/step - loss: 0.0259 - acc: 0.9932 - val_loss: 0.2243 - val_acc: 0.9338

Epoch 00002: val_acc improved from 0.93370 to 0.93380, saving model to /content/gdrive/My Drive/test_9.4L/ep:002-val_acc:0.934.hdf5
Epoch 3/5
50000/50000 [==============================] - 313s 6ms/step - loss: 0.0272 - acc: 0.9927 - val_loss: 0.2250 - val_acc: 0.9334

Epoch 00003: val_acc did not improve from 0.93380
Epoch 4/5
50000/50000 [==============================] - 312s 6ms/step - loss: 0.0263 - acc: 0.9929 - val_loss: 0.2245 - val_acc: 0.9341

Epoch 00004: val_acc improved from 0.93380 to 0.93410, saving model to /content/gdrive/My Drive/test_9.4L/ep:004-val_acc:0.934.hdf5
Epoch 5/5
50000